# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
# import libraries
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import sqlite3
import re
import pickle

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# load data from database
con = sqlite3.connect('DisasterResponse.db')
df = pd.read_sql_query("SELECT * FROM Message", con)
X = df.message
y = df.iloc[:, 6:]

In [3]:
y.columns

Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    # normalizarion
    text = text.lower()
    text = text.strip()

    # Punctuation Removal
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # word_tokenize
    tokens = word_tokenize(text)

    # Remove stop words
    words = [token for token in tokens if token not in stopwords.words("english")]

    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]

    # re-create document from filtered lemmed
    doc = ' '.join(lemmed)
    
    return doc

In [9]:
# test tokenize
normalize_corpus = np.vectorize(tokenize)
norm_corpus = normalize_corpus(X.iloc[:10])
norm_corpus

array(['weather update cold front cuba could pas haiti', 'hurricane',
       'looking someone name',
       'un report leogane 80 90 destroyed hospital st croix functioning need supply desperately',
       'say west side haiti rest country today tonight',
       'information national palace', 'storm sacred heart jesus',
       'please need tent water silo thank',
       'would like receive message thank',
       'croix de bouquet health issue worker santo 15 area croix de bouquet'], 
      dtype='<U87')

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [13]:
print(classification_report(y_test, y_pred, target_names=y.columns))

                        precision    recall  f1-score   support

               request       0.61      0.17      0.26      1092
                 offer       0.00      0.00      0.00        29
           aid_related       0.59      0.39      0.47      2694
          medical_help       0.50      0.01      0.02       514
      medical_products       0.80      0.01      0.02       329
     search_and_rescue       0.00      0.00      0.00       206
              security       0.00      0.00      0.00       129
              military       0.00      0.00      0.00       213
           child_alone       0.00      0.00      0.00         0
                 water       0.65      0.04      0.08       404
                  food       0.61      0.13      0.21       700
               shelter       0.62      0.03      0.06       583
              clothing       1.00      0.01      0.02       100
                 money       1.00      0.01      0.03       148
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'tfidf__max_df': (0.5, 0.75, 1.0),
    'tfidf__ngram_range': ((1, 1), (1,2)),
    'tfidf__max_features': (None, 5000,10000),
    'tfidf__use_idf': (True, False)
}
cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

cv.fit(X_train, y_train)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [27]:
# predict on test data
y_pred = pipeline.predict(X_test)

### 9. Export your model as a pickle file

In [14]:
Pkl_Filename = "message_category.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(pipeline, file)

In [15]:
Pkl_Filename = "message_category.pkl"  

with open(Pkl_Filename, 'rb') as file:  
    Pickled_Model = pickle.load(file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [1]:
! python train_classifier.py messages.csv categories.csv 

MESSAGE Step I: Downlaod need pacakages
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
MESSAGE Step II: Data Wrangling
MESSAGE: Step 1 Load dataSets
MESSAGE: Step 2 Merge the datasets on `id`
MESSAGE: Step 3 Category column transfomation
MESSAGE: Step 4 Add cat_df to df and drop cat col
MESSAGE: Step 5 Export to sql
SUCCESS: Data Wrangling Done
MESSAGE Step III: build model pipeline
MESSAGE Step IV: train model pipeline
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metric